## Collaborative Filtering
The code used here is based on Jeremy Howard's course 'Practical Deep Learning for Coders' and the associated training materials.  
https://www.kaggle.com/code/jhoward/collaborative-filtering-deep-dive/notebook  
  
Collaborative Filtering methods utilize underlying similarities and relationships between user ratings to generate recommendations.  
We use the training data created in the previous step to train the model.

In [108]:
import os
import numpy as np
import pandas as pd

In [109]:
from fastai.collab import *
from fastai.tabular.all import *

In [110]:
file_path = '../data_complete'

In [150]:
#Import Data from the data prep stage
user_ratings_df = pd.read_csv(file_path + '/merged_user_ratings.csv', index_col=0)
book_meta_data = pd.read_csv(file_path + '/book_details.csv', index_col=0)

In [151]:
book_meta_data

,book_id,Name,RatingDist1,pagesNumber,RatingDist4,RatingDistTotal,PublishMonth,PublishDay,Publisher,CountsOfReview,...,Language,Authors,Rating,RatingDist2,RatingDist5,ISBN,RatingDist3,Description,Count of text reviews,PagesNumber
244421,2588957,"!Musica!: Salsa, Rumba, Merengue, & More: The Rhythm of Latin America",1:0,NaN,4:7,total:13,1,1,Diane Pub Co,0,...,eng,Sue Steward,3.69,2:0,5:1,NaN,3:5,"Salsa, the sexy dance music of the Spanish-speaking world, is making its way into the lives of millions around the globe. But the term ""salsa"" actually describes a multitude of different Latin music styles, including Son, Guaracha, Mambo, Rumba, Merengue, &amp; Latin jazz. This book tells the fascinating story of salsa's origins &amp; charts its journey from Cuba to New York's barrio, Miami Beach, &amp; beyond. In more than 50 biographies &amp; portraits, the book celebrates such Latin music legends as Tito Puente &amp; Celia Cruz, the king &amp; queen of salsa; Ruben Blades, one of the mu...",NaN,176.0
4941,911352,""" Memories of Underdevelopment"" and ""Inconsolable Memories"" (Rutgers Films in Print)",1:0,272.0,4:8,total:19,1,6,Rutgers University Press,2,...,eng,Edmundo Desnoes,3.95,2:2,5:6,0813515378,3:3,"Memories of Underdevelopment was the first great international success of Cuban cinema. The film provides a complex portrait of Sergio, a disaffected bourgeois intellectual who remains in Havana after the Revolution, suspended between two worlds. He can no longer accept the values of his family's reactionary past and yet boredom and the conditioning of his early life prevent him from committing himself to the new revolutionary society. Sergio's story is played out in the turbulent period of the Bay of Pigs invasion and the 1962 missile crisis, events he can only watch on his television scr...",2.0,NaN
1468,1003536,"""A Pretty Girl is Like a Melody"" and Other Favorite Song Hits, 1918-1919",1:0,176.0,4:0,total:1,14,4,Dover Publications,0,...,eng,David A. Jasen,2.00,2:1,5:0,0486294218,3:0,"Original sheet music and covers for 40 beloved favorites, including <i>After You've Gone</i>, <i>How Ya Gonna Keep '</i><i>Em Down on the Farm</i>, <i>I'm Always Chasing Rainbows</i>, <i>Rock-a-Bye Your Baby</i>, and 36 other Golden Oldies. For sing-alongs, family get-togethers, parties, and other events.<br />",0.0,NaN
39009,368731,"""A Problem from Hell"": America and the Age of Genocide",1:130,620.0,4:3037,total:8797,6,5,Basic Books,447,...,eng,Samantha Power,4.23,2:317,5:4175,0060541644,3:1138,NaN,NaN,NaN
117360,3329567,"""A"" Is for Aarrgh!",1:1,NaN,4:23,total:46,9,1,HarperCollins Children's Books,1,...,eng,William J. Brooke,3.91,2:3,5:12,0060233931,3:7,"Long, long ago -- even before ""once upon a time"" -- when people lived in caves, and woolly mammoths and saber-tooth tigers roamed the land, communication generally meant a few grunts and a couple of bonk bonks on the head with a big club for emphasis. But what if a young caveboy suddenly began pulling words out of thin air and naming everything in sight? And what if a young cavegirl suddenly took those same words and began weaving them into something magical? Master storyteller William J. Brooke takes readers back to the Stone Age in this whimsical and outrageously funny spoof on the birth...",NaN,249.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
348072,2866335,源氏物語 あさきゆめみし バイリンガル版（星の章）,1:0,NaN,4:8,total:24,7,12,講談社,0,...,eng,Waki Yamato,3.58,2:4,5:5,4770040393,3:7,NaN,NaN,336.0
16754,229340,源氏物語 あさきゆめみし バイリンガル版（花の章下）/ The Tale Of Genji: 2-2,1:0,160.0,4:0,total:0,1,7,講談社,0,...,eng,Waki Yamato,0.00,2:0,5:0,4770027036,3:0,NaN,NaN,NaN
348073,2866338,源氏物語 あさきゆめみし バイリンガル版（花の章）,1:0,NaN,4:4,total:7,7,12,講談社,1,...,eng,Waki Yamato,3.86,2:0,5:1,4770040407,3:2,NaN,NaN,336.0
1242,802223,源氏物語 あさきゆめみし―バイリンガル版（花の章上）/The Tale of Genji: 3,1:0,160.0,4:0,total:2,1,4,講談社,0,...,eng,Waki Yamato,2.00,2:2,5:0,4770027028,3:0,"A rendering in manga of the 11-century epic 

In [112]:
user_ratings_df.rename(columns = {'ID':'user_id', 'Rating_numeric':'rating_num'}, inplace=True)
user_ratings_df.head()

,Unnamed: 0.1,Unnamed: 0,user_id,Name,Rating,rating_num,book_id
0,0.0,0.0,1,Agile Web Development with Rails: A Pragmatic Guide,it was amazing,5.0,45.0
1,1.0,1.0,1,"The Restaurant at the End of the Universe (Hitchhiker's Guide to the Galaxy, #2)",it was amazing,5.0,8695.0
2,2.0,2.0,1,Siddhartha,it was amazing,5.0,171764.0
5,5.0,5.0,1,"The Hunger Games (The Hunger Games, #1)",it was amazing,5.0,2767052.0
6,6.0,6.0,1,"The Clue in the Embers (Hardy Boys, #35)",it was amazing,5.0,76932.0


In [113]:
user_ratings_df_select = user_ratings_df[['user_id','book_id','rating_num']]
dls = CollabDataLoaders.from_df(user_ratings_df_select, user_name='user_id', item_name='book_id', rating='rating_num')
dls.show_batch()

,user_id,book_id,rating_num
0,7268,46069.0,4.0
1,10627,69242.0,3.0
2,4086,3431.0,4.0
3,5,4101648.0,4.0
4,3593,2767052.0,5.0
5,8029,14201.0,5.0
6,545,5.0,4.0
7,3898,4631.0,5.0
8,8610,12321.0,4.0
9,10969,439288.0,4.0


In [114]:
#Fit the collab_learner
learn = collab_learner(dls, y_range=(0,5.5), n_factors=100, layers=[100,50])
'''
dls is the data loader
The expected range of y values is specified between 0 and 5.5. This is done (even though a user rating cant exceed 5) 
since practically we would actually expect to get rating values of 5 in the data as well. if the limits are specified as 0-5, using
a sigmoid implementation it would be unlikely to actually see a rating of 5 in the results.

The structure of the learner has been set after some trial and error and is not the optimal configuration. Further hyperparameter
tuning is left for a future exercise.

'''
learn.fit_one_cycle(1, 5e-3, wd=0.01)

epoch,train_loss,valid_loss,time
0,0.643558,0.958121,02:11


In [171]:
# export the model
# torch.save(dls, 'goodread_data_loader.pkl')
# learn.export('goodread_rec.pkl')

In [172]:
dls_load = torch.load('goodread_data_loader.pkl')
learn_load = load_learner('goodread_rec.pkl')
learn_load.model

EmbeddingDotBias(
  (u_weight): Embedding(3341, 100)
  (i_weight): Embedding(25051, 100)
  (u_bias): Embedding(3341, 1)
  (i_bias): Embedding(25051, 1)
)

In [173]:
# create a function that recommends books using the item weights
def similar_books(books):
    # Get the Latent Factors (Weight) of the books
    book_factors = learn_load.model.i_weight.weight
    
    idx = dls_load.classes['book_id'].o2i[books]
    distances = nn.CosineSimilarity(dim=1)(book_factors, book_factors[idx][None])
    t = distances.argsort(descending=True)[:6]
    t = t[t!=t[0]]
    reco_list = list(dls_load.classes['book_id'][t])
    book_list = [return_bookname_from_id(i) for i in reco_list]
    
    return book_list

In [166]:
def return_bookname_from_id(book_id):
    try:
        return book_meta_data.loc[book_meta_data['book_id']==book_id]['Name'].values[0]
    except (KeyError, IndexError, Exception) as ex:
        return 'Book not found'

In [174]:
return_bookname_from_id(171764)

'Siddhartha'

In [175]:
similar_books(171764)

['Gold: The Final Science Fiction Collection',
 'Later Novels and Other Writings: The Lady in the Lake / The Little Sister / The Long Goodbye / Playback / Double Indemnity (screenplay) / Selected Essays and Letters',
 'Child of God',
 'Burning Chrome',
 'The Serpent and the Rainbow']

In [126]:
user_ratings_df.loc[user_ratings_df['book_id']== 41832.0]['Name'].unique()

array(['Gold: The Final Science Fiction Collection'], dtype=object)

In [102]:
user_ratings_df.head()

,Unnamed: 0,user_id,Name,Rating,rating_num,book_id
0,0.0,1,Agile Web Development with Rails: A Pragmatic Guide,it was amazing,5.0,45.0
1,1.0,1,"The Restaurant at the End of the Universe (Hitchhiker's Guide to the Galaxy, #2)",it was amazing,5.0,8695.0
2,2.0,1,Siddhartha,it was amazing,5.0,171764.0
6,6.0,1,"The Clue in the Embers (Hardy Boys, #35)",it was amazing,5.0,76932.0
7,7.0,1,The Authoritative Calvin and Hobbes: A Calvin and Hobbes Treasury,it was amazing,5.0,59715.0
